# Link to the video for column tranformer :- https://www.youtube.com/watch?v=5TVj6iEBR4I

In [78]:
import pandas as pd 


In [79]:
df = pd.read_csv('covid_toy.csv')

In [80]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


In [82]:
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer



In [83]:
df.gender.value_counts()

gender
Female    59
Male      41
Name: count, dtype: int64

In [84]:
df.cough.value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [85]:
df.city.value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [86]:
df.fever.info()

<class 'pandas.core.series.Series'>
RangeIndex: 100 entries, 0 to 99
Series name: fever
Non-Null Count  Dtype  
--------------  -----  
90 non-null     float64
dtypes: float64(1)
memory usage: 932.0 bytes


In [87]:
df.shape

(100, 6)

In [88]:
df.fever.isnull().sum()

np.int64(10)

In [89]:
## So gender, city are nominal categorical variables so i will use onehotEncoder for them.
## Cough is ordinal categorical variable so i will use ordinal encoder for it 
## fever which has 10 values missing values i will be using SimpleImputer mean to fill those missing values 


In [90]:
transformer = ColumnTransformer(transformers=[
              ('imputer', SimpleImputer(strategy = 'mean'),['fever']),
              ('onehot',OneHotEncoder(sparse_output=False, drop='first'),['gender','city'] ),
              ('ordinal', OrdinalEncoder(categories=[['Mild','Strong']]), ['cough'])
              ]  , remainder= 'passthrough',                    
                               
                               )

transformer.set_output(transform = 'pandas')

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('imputer', ...), ('onehot', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``fea

In [91]:
## hence we have creadted a column transformer now we have to train test split the data and apply the transformations

In [92]:
from sklearn.model_selection import train_test_split

df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [93]:
x_train,x_test,y_train,y_test = train_test_split(df.drop('has_covid',axis = 1), df['has_covid'], test_size =0.2, random_state=42)

In [99]:

x_train_transformered = transformer.fit_transform(x_train)
x_train_transformered.head()

,imputer__fever,onehot__gender_Male,onehot__city_Delhi,onehot__city_Kolkata,onehot__city_Mumbai,ordinal__cough,remainder__age
55,101.0,0.0,0.0,0.0,1.0,0.0,81
88,100.0,0.0,0.0,1.0,0.0,0.0,5
26,100.0,0.0,0.0,1.0,0.0,0.0,19
42,100.0,1.0,1.0,0.0,0.0,0.0,27
69,103.0,0.0,1.0,0.0,0.0,0.0,73


In [95]:
transformer.transform(x_test).shape

(20, 7)

In [96]:
## let us create one more column transformer for the target column and use label encoder for it 
from sklearn.preprocessing import LabelEncoder

In [97]:
tnf1 = ColumnTransformer(transformers=[
    ('label',LabelEncoder(), ['has_covid'])
], remainder='passthrough')
tnf1.fit_transform(y_train).shape
 
 ## this  wil throw and error because
 
#  LabelEncoder is meant for encoding target labels and should not be used inside ColumnTransformer, which is meant for feature preprocessing. Instead, you should use LabelEncoder separately to encode the target variable before fitting your model. Here's how you can do it:

IndexError: tuple index out of range

In [ ]:
le = LabelEncoder()
y_train_new = le.fit_transform(y_train)
y_train_new


array([1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1])